## Libaries required installation

In [ ]:
!pip install -q pypdf sentence-transformers faiss-cpu transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 45.9 MB/s eta 0:00:00


In [ ]:
import os
from pathlib import Path

DOCS_DIR = Path("./docs")
DOCS_DIR.mkdir(parents=True, exist_ok=True)

print(f"PDF storage folder ready at: {DOCS_DIR.resolve()}")

# this is the snippet that creates the KB (knowledge base) folder and ready for extreaction

PDF storage folder ready at: /content/docs


## Upload PDFs to the KB container

In [ ]:
from pathlib import Path
import sys

DOCS_DIR = Path("./docs")
IN_COLAB = "google.colab" in sys.modules


if IN_COLAB:
    from google.colab import files
    print("Please select 2–3 small PDFs…")
    uploaded = files.upload()  # opens the file chooser
    for fname, data in uploaded.items():
        (DOCS_DIR / fname).write_bytes(data)
else:
    print(f"Running locally; make sure your PDFs are in: {DOCS_DIR.resolve()}")

Please select 2–3 small PDFs…


Saving awsmp_Matillion_ML_eBook_092019.pdf to awsmp_Matillion_ML_eBook_092019.pdf
Saving Machine Learning for Absolute Beginners.pdf to Machine Learning for Absolute Beginners.pdf
Saving human-rights-booklet.pdf to human-rights-booklet.pdf


In [ ]:
pdf_paths = sorted(DOCS_DIR.glob("*.pdf")) # -> gathering the PDFs

In [ ]:
n = len(pdf_paths)
total_mb = sum(p.stat().st_size for p in pdf_paths) / (1024 * 1024)

print(f"\nFound {n} PDF(s) in {DOCS_DIR.resolve()}:")
for p in pdf_paths:
    print(f" - {p.name}  ({p.stat().st_size/1024:.1f} KB)")

assert 2 <= n <= 3, f"Please upload 2–3 PDFs (currently {n})."
assert total_mb <= 30, f"Total size {total_mb:.2f} MB is large; keep under ~30 MB for a quick demo."

print(f"\n PDFs registered. Total size: {total_mb:.2f} MB")

# verfying the pdf are in the folder and the size constrains of < 30 mb


Found 3 PDF(s) in /content/docs:
 - Machine Learning for Absolute Beginners.pdf  (11616.3 KB)
 - awsmp_Matillion_ML_eBook_092019.pdf  (10014.1 KB)
 - human-rights-booklet.pdf  (2697.4 KB)

 PDFs registered. Total size: 23.76 MB


## Pre Chunking , Data Cleanser & Pre processor

In [ ]:
from pathlib import Path
from pypdf import PdfReader
from dataclasses import dataclass, asdict
import re
import json

DOCS_DIR = Path("./docs")
pdf_paths = sorted(DOCS_DIR.glob("*.pdf"))

# pdfreader for extracting text based only to prevent OCRs and dataclasses to ease the record type for each page with an easy JSON export , regex for text cleanup save the extracted pages

In [ ]:
@dataclass
class PageRecord:
    doc_id: str        # filename without extension
    source_path: str   # file path
    page_num: int      # 1-based page index
    text: str          # raw extracted text (lightly cleaned)



# this is an approach that store each page as structured object to know where each chunk came from , for processing later (chunks , embedding)

In [ ]:
def clean_text(s: str) -> str:
    if not s:
        return ""
    s = s.replace("\x00", " ")            # remove nulls if any
    s = re.sub(r"[ \t]+", " ", s)         # collapse spaces/tabs
    s = re.sub(r"\n\s+\n", "\n\n", s)     # collapse blank lines
    s = s.strip()
    return s


    # removing anything distract the way of retrieval withpout reflowing process

In [ ]:
def extract_pdf_pages(pdf_path: Path) -> list[PageRecord]:
    """Extract plain text from each page of a PDF (no OCR)."""
    reader = PdfReader(str(pdf_path))
    pages: list[PageRecord] = []
    for i, page in enumerate(reader.pages):
        raw = page.extract_text()  # returns None for image only pages
        text = clean_text(raw or "")
        pages.append(PageRecord(
            doc_id=pdf_path.stem,
            source_path=str(pdf_path),
            page_num=i + 1,
            text=text
        ))
    return pages

    # this is the retrieved infromation label for citations , looping through pages extracting only raw text cleans and captures it :

    # ( doc_id -> without .pdf ), (source_path -> file location) , (page_num -> page number) , (text - > cleaned text)

In [ ]:
corpus_pages: list[PageRecord] = []
for path in pdf_paths:
    corpus_pages.extend(extract_pdf_pages(path))

    # flattens all docs pages into a list so we can later downstream

In [ ]:
# checking how will the extractor work and remove any unnecassary texts or images
total_pages = len(corpus_pages)
empty_pages = sum(1 for r in corpus_pages if len(r.text) == 0)
print(f"Extracted {total_pages} page(s) from {len(pdf_paths)} document(s).")
print(f"Empty pages (likely images/figures only): {empty_pages}")


Extracted 260 page(s) from 3 document(s).
Empty pages (likely images/figures only): 37


In [ ]:
# testing the process and preview our clean data
for rec in corpus_pages[:3]:
    print(f"\n[{rec.doc_id} :: page {rec.page_num}]")
    preview = rec.text[:500]
    print(preview + ("..." if len(rec.text) > 500 else ""))


[Machine Learning for Absolute Beginners :: page 1]


[Machine Learning for Absolute Beginners :: page 2]


[Machine Learning for Absolute Beginners :: page 3]
Machine Learning
For
Absolute Beginners:
A Plain English Introduction
Third Edition
Oliver Theobald


Validation and debug step , JSON format to check extraction worked befor feeding to the chunking level in case of bad retrieval

In [ ]:
with open("kb_pages.jsonl", "w", encoding="utf-8") as f:
    for rec in corpus_pages:
        json.dump(asdict(rec), f, ensure_ascii=False)
        f.write("\n")

print("\n Saved page-level extraction to kb_pages.jsonl (one JSON object per page).")


 Saved page-level extraction to kb_pages.jsonl (one JSON object per page).


In [ ]:
import pandas as pd

records = []
with open("kb_pages.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        records.append(json.loads(line))

df = pd.DataFrame(records)
df.head(5)

,doc_id,source_path,page_num,text
0,Machine Learning for Absolute Beginners,docs/Machine Learning for Absolute Beginners.pdf,1,
1,Machine Learning for Absolute Beginners,docs/Machine Learning for Absolute Beginners.pdf,2,
2,Machine Learning for Absolute Beginners,docs/Machine Learning for Absolute Beginners.pdf,3,Machine Learning\nFor\nAbsolute Beginners:\nA ...
3,Machine Learning for Absolute Beginners,docs/Machine Learning for Absolute Beginners.pdf,4,Third Edition\nCopyright © 2021 by Oliver Theo...
4,Machine Learning for Absolute Beginners,docs/Machine Learning for Absolute Beginners.pdf,5,TABLE OF CONTENTS\nPREFACE\nWHAT IS MACHINE LE...


## Text chunking for Embedding

In [ ]:
from typing import List, Dict

@dataclass
class ChunkRecord:
    doc_id: str
    source_path: str
    page_num: int
    chunk_id: int
    text: str

# each chunk keeps the metadata for each belongings (units of knowledge that will be embedded)

In [ ]:
def chunk_text(text: str, chunk_size: int = 500, overlap: int = 100) -> List[str]:
    words = text.split()
    if not words:
        return []

    chunks = []
    start = 0
    while start < len(words):
        end = start + chunk_size
        chunk_words = words[start:end]
        chunks.append(" ".join(chunk_words))
        if end >= len(words):
            break
        start = end - overlap
    return chunks

    # input will be text on one page(string)
    # Step 1 (split into words)
    #        |__ Step 2 chunck size words at a time
    #            |______ Step 3 moving the sliding window forward but rewind by the overlap words before the next chunk --> Chunk_1 = words [0 - 499] , Chunk _2 = words [400 - 899] : overlaps by 100
    # output list chunked strings for embedding


In [ ]:
corpus_chunks: List[ChunkRecord] = []
for rec in corpus_pages:
    chunks = chunk_text(rec.text, chunk_size=500, overlap=100)
    for i, ch in enumerate(chunks):
        corpus_chunks.append(ChunkRecord(
            doc_id=rec.doc_id,
            source_path=rec.source_path,
            page_num=rec.page_num,
            chunk_id=i,
            text=ch
        ))


        # loops through all the pages recorded from the extracted step and applies the chunking
        #                     |____________ runs the chunk text on its record object text
        #                                   |___________ wraps each chunk into Chunk record with its meta data
        #                                                |________ appends to the the full list of all chunks accross all pdfs and files

In [ ]:
print(f"Created {len(corpus_chunks)} chunks from {len(corpus_pages)} pages.")

# total chunks created from the extracted pages

Created 226 chunks from 260 pages.


In [ ]:
for rec in corpus_chunks[:3]:
    print(f"\n[{rec.doc_id} :: page {rec.page_num} :: chunk {rec.chunk_id}]")
    print(rec.text[:400] + ("..." if len(rec.text) > 400 else ""))


    # preview of the chunks checking reasonable informations extracted


[Machine Learning for Absolute Beginners :: page 3 :: chunk 0]
Machine Learning For Absolute Beginners: A Plain English Introduction Third Edition Oliver Theobald

[Machine Learning for Absolute Beginners :: page 4 :: chunk 0]
Third Edition Copyright © 2021 by Oliver Theobald All rights reserved. No part of this publication may be reproduced, distributed, or transmitted in any form or by any means, including photocopying, recording, or other electronic or mechanical methods, without the prior written permission of the publisher, except in the case of brief quotations embodied in critical reviews and certain other non-co...

[Machine Learning for Absolute Beginners :: page 5 :: chunk 0]
TABLE OF CONTENTS PREFACE WHAT IS MACHINE LEARNING? MACHINE LEARNING CATEGORIES THE MACHINE LEARNING TOOLBOX DATA SCRUBBING SETTING UP YOUR DATA LINEAR REGRESSION LOGISTIC REGRESSION K -NEAREST NEIGHBORS K -MEANS CLUSTERING BIAS & VARIANCE SUPPORT VECTOR MACHINES ARTIFICIAL NEURAL NETWORKS DECISION TREE

## Embedding chunks & building FAISS index

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

In [ ]:
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
chunk_texts = [rec.text for rec in corpus_chunks]
print(f"Generating embeddings for {len(chunk_texts)} chunks...")

Generating embeddings for 226 chunks...


In [ ]:
embeddings = embed_model.encode(chunk_texts, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)

print("✅ Embedding shape:", embeddings.shape)  # (num_chunks, dim

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

✅ Embedding shape: (226, 384)


In [ ]:
dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)  # IP = inner product (works as cosine sim since we normalized)
index.add(embeddings)

print(f"✅ FAISS index built with {index.ntotal} vectors.")

✅ FAISS index built with 226 vectors.


In [ ]:
chunk_metadata = corpus_chunks

In [ ]:
texts = chunk_texts

## Testing retreival with FAISS

In [ ]:
query = "What is machine learning?"

In [ ]:
query_vec = embed_model.encode([query], convert_to_numpy=True, normalize_embeddings=True) # encoding the vector into numbers for the model , normalizing the vector len = 1 for ease similarity checks

In [ ]:
k = 3  # how many chunks to retrieve
scores, indices = index.search(query_vec, k)

In [ ]:
print(f"\n Query: {query}\n")
for rank, idx in enumerate(indices[0]):
    meta = chunk_metadata[idx]
    score = scores[0][rank]
    print(f"Result {rank+1} (score={score:.3f})")
    print(f" - Doc: {meta.doc_id}, Page: {meta.page_num}, Chunk: {meta.chunk_id}")
    print(f" - Preview: {meta.text[:300]}...\n")


 Query: What is machine learning?

Result 1 (score=0.744)
 - Doc: Machine Learning for Absolute Beginners, Page: 10, Chunk: 0
 - Preview: 2 WHAT IS MACHINE LEARNING? In 1959, IBM published a paper in the IBM Journal of Research and Development with an intriguing and obscure title. Authored by IBM’s Arthur Samuel, the paper investigated the application of machine learning in the game of checkers “to verify the fact that a computer can ...

Result 2 (score=0.704)
 - Doc: awsmp_Matillion_ML_eBook_092019, Page: 3, Chunk: 0
 - Preview: Introduction Industry experts, competitors and even your customers are talking about machine learning and artificial intelligence. The terms, while used widely and interchangeably, are often misunderstood and carry a narrow definition. Both machine learning and artificial intelligence have distinct ...

Result 3 (score=0.685)
 - Doc: Machine Learning for Absolute Beginners, Page: 15, Chunk: 0
 - Preview: Figure 4: Visual representation of the relationship be

## Answer generation RAG - test_1 (Using flan-t5-base)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

In [ ]:
gen_model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(gen_model_name)
gen_model = AutoModelForSeq2SeqLM.from_pretrained(gen_model_name)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def rag_answer(query, top_k=3, max_new_tokens=200):
    # Embed the query
    query_vec = embed_model.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    scores, indices = index.search(query_vec, top_k)

    # Gather retrieved context
    retrieved_chunks = []
    for idx in indices[0]:
        meta = chunk_metadata[idx]
        retrieved_chunks.append(meta.text)
    context = "\n".join(retrieved_chunks)

    # Build the prompt
    prompt = (
        f"Answer the following question using only the provided context.\n\n"
        f"Context:\n{context}\n\n"
        f"Question: {query}\nAnswer:"
    )

    # Tokenize and generate
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = gen_model.generate(**inputs, max_new_tokens=max_new_tokens)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer, retrieved_chunks

In [ ]:
query = "What is machine learning?"
answer, used_chunks = rag_answer(query, top_k=3)

print("🔎 Question:", query)
print("\n📖 Retrieved context (first chunk preview):")
print(used_chunks[0][:400] + "...\n")
print("🤖 Answer:", answer)

🔎 Question: What is machine learning?

📖 Retrieved context (first chunk preview):
2 WHAT IS MACHINE LEARNING? In 1959, IBM published a paper in the IBM Journal of Research and Development with an intriguing and obscure title. Authored by IBM’s Arthur Samuel, the paper investigated the application of machine learning in the game of checkers “to verify the fact that a computer can be programmed so that it will learn to play a better game of checkers than can be played by the pers...

🤖 Answer: ad revenue


## Answer generation RAG - test_2 (Using gpt)

In [ ]:
import numpy as np
from openai import OpenAI

client = OpenAI(
    api_key="API_KEY",
     base_url="Your_API_URL"
     )

In [ ]:
def rag_answer(query, index, embeddings, texts, k=3):
    # Create query embedding
    q_emb = client.embeddings.create(
        model="openai/text-embedding-3-small",
        input=[query]
    ).data[0].embedding

    # Search FAISS index
    D, I = index.search(np.array([q_emb], dtype="float32"), k)
    retrieved = [texts[i] for i in I[0]]

    # Build context
    context = "\n\n".join(retrieved)

    # Ask GPT with retrieved context
    prompt = f"""You are an AI assistant.
Use the context below to answer the question.
If the answer is not in the documents, say "I could not find an answer in the documents."

Context:
{context}

Question: {query}
Answer with citations from the context.
"""
    response = client.chat.completions.create(
        model="openai/gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content


In [ ]:
print(rag_answer("What is machine learning?", index, embeddings, texts))

AttributeError: 'str' object has no attribute 'data'

## Answer generation RAG test_3 (using DistilBERT)

In [ ]:
from transformers import pipeline


qa_model = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
def rag_answer_distilbert(query, index, embeddings, texts, k=3):
    # Retrieve top-k chunks from FAISS
    q_emb = embed_model.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    D, I = index.search(q_emb, k)
    retrieved = [texts[i] for i in I[0]]

    # Pick the best chunk using DistilBERT QA
    answers = []
    for ctx in retrieved:
        result = qa_model(question=query, context=ctx)
        answers.append((result["score"], result["answer"], ctx[:200]))

    # Return the best scoring answer
    best = max(answers, key=lambda x: x[0])
    return {
        "answer": best[1],
        "confidence": best[0],
        "context_preview": best[2]
    }

In [ ]:
print(rag_answer_distilbert("What is machine learning?", index, embeddings, texts))

{'answer': 'the process of building and training models to process data', 'confidence': 0.44814351201057434, 'context_preview': 'Introduction Industry experts, competitors and even your customers are talking about machine learning and artificial intelligence. The terms, while used widely and interchangeably, are often misunders'}


In [ ]:
# multiple answers ranked by confidence

def rag_answer_distilbert_ranked(query, index, texts, metadata, k=3):
    # Retrieve top-k chunks
    q_emb = embed_model.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    D, I = index.search(q_emb, k)
    retrieved = [(texts[i], metadata[i]) for i in I[0]]

    answers = []
    for ctx, meta in retrieved:
        result = qa_model(question=query, context=ctx)
        answers.append({
            "answer": result["answer"],
            "confidence": float(result["score"]),
            "doc": meta.doc_id,
            "page": meta.page_num,
            "preview": ctx[:200] + "..."
        })

    # Sort by confidence descending
    answers = sorted(answers, key=lambda x: x["confidence"], reverse=True)
    return answers


In [ ]:
results = rag_answer_distilbert_ranked("What is machine learning?", index, texts, chunk_metadata, k=3)
for r in results:
    print(f"Answer: {r['answer']} (Conf: {r['confidence']:.2f}) [Doc: {r['doc']}, Page: {r['page']}]")
    print("Preview:", r["preview"])
    print("----")

Answer: the process of building and training models to process data (Conf: 0.45) [Doc: awsmp_Matillion_ML_eBook_092019, Page: 3]
Preview: Introduction Industry experts, competitors and even your customers are talking about machine learning and artificial intelligence. The terms, while used widely and interchangeably, are often misunders...
----
Answer: it provides a narrower and more practical lens of study (Conf: 0.22) [Doc: Machine Learning for Absolute Beginners, Page: 15]
Preview: Figure 4: Visual representation of the relationship between data-related fields For students interested in AI, machine learning provides an excellent starting point as it provides a narrower and more ...
----
Answer: a subfield of computer science (Conf: 0.07) [Doc: Machine Learning for Absolute Beginners, Page: 10]
Preview: 2 WHAT IS MACHINE LEARNING? In 1959, IBM published a paper in the IBM Journal of Research and Development with an intriguing and obscure title. Authored by IBM’s Arthur Samuel, the pa

hybrid approach with generative layer (Flan-T5)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


gen_model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(gen_model_name)
gen_model = AutoModelForSeq2SeqLM.from_pretrained(gen_model_name).to("cuda")

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def rag_answer_hybrid(query, index, texts, metadata, k=3, max_new_tokens=250):
    q_emb = embed_model.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    D, I = index.search(q_emb, k)
    retrieved = [(texts[i], metadata[i]) for i in I[0]]

    extractive_answers = []
    for ctx, meta in retrieved:
        result = qa_model(question=query, context=ctx)
        extractive_answers.append({
            "answer": result["answer"],
            "confidence": float(result["score"]),
            "doc": meta.doc_id,
            "page": meta.page_num,
            "preview": ctx[:200] + "..."
        })

    context_with_meta = ""
    for ctx, meta in retrieved:
        context_with_meta += f"[Doc: {meta.doc_id}, Page: {meta.page_num}]\n{ctx}\n\n"

    prompt = f"""You are an expert AI tutor.
Using the context, explain the answer to the question in detail.
Write 3–5 full sentences in your own words.
Do not just repeat the context.
Always end with sources in the format (Doc name, Page number).

Context:
{context_with_meta}

Question: {query}
Answer:"""

    #GPU force use
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to("cuda")
    outputs = gen_model.generate(**inputs, max_new_tokens=max_new_tokens)
    generative_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if len(generative_answer.split()) < 10:
        generative_answer = " / ".join([r['answer'] for r in extractive_answers])

    return {
        "extractive_answers": extractive_answers,
        "generative_answer": generative_answer
    }

In [ ]:
result = rag_answer_hybrid("What is machine learning?", index, texts, chunk_metadata, k=3)

print("🔎 Extractive Answers:")
for r in result["extractive_answers"]:
    print(f"- {r['answer']} (Conf: {r['confidence']:.2f}) [Doc: {r['doc']}, Page: {r['page']}]")

print("\n🤖 Generative Answer:")
print(result["generative_answer"])

🔎 Extractive Answers:
- a subfield of computer science (Conf: 0.07) [Doc: Machine Learning for Absolute Beginners, Page: 10]
- the process of building and training models to process data (Conf: 0.45) [Doc: awsmp_Matillion_ML_eBook_092019, Page: 3]
- it provides a narrower and more practical lens of study (Conf: 0.22) [Doc: Machine Learning for Absolute Beginners, Page: 15]

🤖 Generative Answer:
Artificial intelligence to act more human, apply learnings in an automated manner.


**the hybrid approach took 1 minute to execute but answered in a good expected way since am limited in time to enhance it more because the model used considered (770M parameters) CPU constraints so i forced it to use the CUDA provided from colab to see a 1 second exectution which is perfect**

## Answer generation RAG final_test (using DistilBERT & Flan-T5)

In [ ]:
def rag_answer_hybrid_final(query, index, texts, metadata, k=3, max_new_tokens=400):
    import re

    # Step 1: Retrieve top-k chunks with FAISS
    q_emb = embed_model.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    D, I = index.search(q_emb, k)

    # Step 2: Deduplicate near-identical chunks
    seen = set()
    retrieved = []
    for i in I[0]:
        chunk = texts[i]
        if chunk[:80] not in seen:
            seen.add(chunk[:80])
            retrieved.append((chunk, metadata[i]))

    # Step 3: DistilBERT extractive answers
    extractive_answers = []
    for ctx, meta in retrieved:
        result = qa_model(question=query, context=ctx)
        extractive_answers.append({
            "answer": result["answer"],
            "confidence": float(result["score"]),
            "doc": meta.doc_id,
            "page": meta.page_num,
            "preview": ctx[:200] + "..."
        })

    # Step 4: Build formatted context for Flan-T5
    context_with_meta = ""
    for ctx, meta in retrieved:
        context_with_meta += f"[Doc: {meta.doc_id}, Page: {meta.page_num}]\n{ctx}\n\n"

    # Step 5: Prompt Flan-T5 to generate clean, cited, 2-paragraph explanation
    prompt = f"""
You are a helpful and professional AI tutor.

Please answer the following question using ONLY the context below.
Paraphrase everything — do NOT copy the text.
Write two full paragraphs that explain the answer clearly.
Cite your sources inline using the format (Doc: name, Page: number).

Context:
{context_with_meta}

Question: {query}

Answer:
"""

    # Step 6: Run Flan-T5 with strong decoding settings
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to("cuda")
    outputs = gen_model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        min_new_tokens=100,
        do_sample=False,
        num_beams=4,
        no_repeat_ngram_size=4,
        repetition_penalty=1.2,
        early_stopping=True
    )
    generative_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Step 7: Return raw model output (even if it's broken)
    return {
        "extractive_answers": extractive_answers,
        "generative_answer": generative_answer
    }


In [ ]:
query = "What is machine learning?"
result = rag_answer_hybrid_final(query, index, texts, chunk_metadata)

print("\n🔎 Extractive Answers:")
for r in result["extractive_answers"]:
    print(f"- {r['answer']} (Conf: {r['confidence']:.2f}) [Doc: {r['doc']}, Page: {r['page']}]")

print("\n🤖 Generative Answer:\n")
print(result["generative_answer"])


🔎 Extractive Answers:
- a subfield of computer science (Conf: 0.07) [Doc: Machine Learning for Absolute Beginners, Page: 10]
- the process of building and training models to process data (Conf: 0.45) [Doc: awsmp_Matillion_ML_eBook_092019, Page: 3]
- it provides a narrower and more practical lens of study (Conf: 0.22) [Doc: Machine Learning for Absolute Beginners, Page: 15]

🤖 Generative Answer:

[Doc: Machine Learning for Absolute Beginners, Page: 10] 2 WHAT IS MACHINE LEARNING? In 1959, IBM published a paper in the IBM Journal of Research and Development with an intriguing and obscure title. Authored by IBM’s Arthur Samuel, the paper investigated the application of machine learning in the game of checkers “to verify the fact that a computer can be programmed so that it will learn to play a better game of Checkers than can be played by the person who wrote the program.” [5] Figure 1: Historical mentions of “machine learning” in published books. Source: Google Ngram Viewer, 2017


In [ ]:
import pandas as pd
from IPython.display import display, Markdown

# Test query
query = "What is machine learning?"
result = rag_answer_hybrid_final(query, index, texts, chunk_metadata, k=3, max_new_tokens=400)

# Format extractive answers into a DataFrame
extractive_df = pd.DataFrame(result["extractive_answers"])
extractive_df = extractive_df[["answer", "confidence", "doc", "page"]]  # keep main cols
extractive_df["confidence"] = extractive_df["confidence"].round(2)

# Display Extractive Answers
display(Markdown("### 🔎 Extractive Answers (Top Chunks Retrieved)"))
display(extractive_df)

# Display Generative Answer
display(Markdown("### 🤖 Generative Answer"))
display(Markdown(result["generative_answer"]))

### 🔎 Extractive Answers (Top Chunks Retrieved)

,answer,confidence,doc,page
0,a subfield of computer science,0.07,Machine Learning for Absolute Beginners,10
1,the process of building and training models to...,0.45,awsmp_Matillion_ML_eBook_092019,3
2,it provides a narrower and more practical lens...,0.22,Machine Learning for Absolute Beginners,15


### 🤖 Generative Answer

AI is the process of building and training models to process data. In this capacity, your models are learning from your data to make predictions. Machine learning, therefore, is the process of building and training models to process data. In this capacity, your models are learning from your data to make predictions. Artificial intelligence, therefore, is the process of building and training models to process data. In this capacity, your models are learning from your data to make predictions.